## Calculates sea level at the Dutch coast as a result of mass changes in Greenland and Antarctica.
The results are saved in a netcdf file

In [2]:
# Load the relevant packages: 

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import xarray as xr

# Package for regridding 
import xesmf as xe 

# Package to make nice plots with different projections 
import cartopy.crs as ccrs 

# Toolkit for  the sea level equation
import gravity_toolkit as gravtk

#from scipy import signal              
import importlib

# Functions to make the script more readable. 
import GravisFunctions as gf

import sys
sys.path.append('../code')
import SeaLevelContrib as slc


importlib.reload(slc)
importlib.reload(gf)

<module 'GravisFunctions' from 'C:\\Users\\kyrab\\Github\\SLBudget\\notebooks\\GravisFunctions.py'>

### Load the datasets:

Gridded ice mass changes from GravIS [1] based on COST-G [2] can be downloaded from the website (https://gravis.gfz.de/). 

Antarctica: 
https://isdc-data.gfz.de/grace/GravIS/COST-G/Level-3/ICE/AIS/. Then download the netcdf file. 

Greenland: 
https://isdc-data.gfz.de/grace/GravIS/COST-G/Level-3/ICE/AIS/. Also download the netcdf file. 


<div style="font-size:0.7em; line-height:1.25;">

[1]: Dahle, C., Boergens, E., Sasgen, I., Döhne, T., Reißland, S., Dobslaw, H.,  
      Klemann, V., Murböck, M., König, R., Dill, R., Sips, M., Sylla, U., Groh, A.,  
      Horwath, M., and Flechtner, F. (2025). *GravIS: mass anomaly products from satellite gravimetry*.  
      **Earth System Science Data**, 17, 611–631. https://doi.org/10.5194/essd-17-611-2025

[2]: Sasgen, I., Groh, A., and Horwath, M. (2020). *COST-G GravIS RL01 Ice-Mass Change Products* (V. 0003).  
      GFZ Data Services. https://doi.org/10.5880/COST-G.GRAVIS_01_L3_ICE

</div>

Landsea mask: Available on request.  


In [3]:
###  Define the path to open the data
data_path = '../data/GravIs/'
output_path = '../outputs/Gravis_Results/'
path_masks = '/Users/kyrab/Github/SLBudget/data/Masks/'

In [4]:
# Open the landsea mask and create a combined mask used in the regridding process: 
masks_ds = xr.open_dataset(f'{path_masks}reference_masks.nc')
landsea = xr.where(masks_ds.mask==1, 0, 1)

In [5]:
# Open the datasets and create yearly data: 

#Antarctica: 
ant = xr.open_dataset(data_path + "GRAVIS-3_COSTG_0100_AIS_GRID_TUD_0003.nc")

#If you want first resample to monthly data and then interpolate it can be done like this, however, this resulted in less accurate results. 
#ant = ant.resample(time="M").interpolate("linear")

ant_y = gf.yearlygrace(ant) 

#Greenland: 
gre = xr.open_dataset(data_path + "GRAVIS-3_COSTG_0100_GIS_GRID_TUD_0003.nc")
#gre = gre.resample(time="M").interpolate("linear")

gre_y = gf.yearlygrace(gre) 


In [6]:
#Regrid the data to match lon lat grid of the landsea mask to use it in the sea level equaton. Yearly data is used so only yearly data needs to be regridded. 

# Use the lon and lat values from the mask directly: Here, points are defined between 2 degrees. It contains for lat all values 89.5 - n with n in N
lon_target = masks_ds['lon'].values
lat_target = masks_ds['lat'].values 

#create lon lat pairs
lon2d, lat2d = np.meshgrid(lon_target, lat_target)   

# again lon lat paris for the regridding. 
ds_out = xr.Dataset(
    {
        "lat": (["lat", "lon"], lat2d),
        "lon": (["lat", "lon"], lon2d)
    }
)

# First make sure nans are filled with zeros. Do that before regridding in stead of after. Otherwise some areas at the boundary are missed and at the boundaries (mainly at antarctica) most mass loss happens. 
ant_y['dm'] = ant_y['dm'].fillna(0)
gre_y['dm'] = gre_y['dm'].fillna(0)

# Create the regridder. Array dataset containin lat lon values. 
regridder = xe.Regridder(ant_y, ds_out, "bilinear", periodic=False, reuse_weights=False, unmapped_to_nan=True)  # Periodic true does not work for some reason in this region... you need apparently full latitude coverage
regridder_gre = xe.Regridder(gre_y, ds_out, "bilinear", periodic=False, reuse_weights=False, unmapped_to_nan=True)

# Regrid dm to lon-lat grid
dm_lonlat = regridder(ant_y["dm"], keep_attrs=True)
dm_lonlat_gre = regridder_gre(gre_y["dm"], keep_attrs=True)
      
dm_lonlat_zero = dm_lonlat.fillna(0) 
dm_lonlat_zero_gre = dm_lonlat_gre.fillna(0) 

In [7]:
# Create a mask of the regridded area of antarctica to make sure all gridded data is masked as land. Combine the ones for greenland and antarctica in one mask called maskje. 

totalant_withnan = dm_lonlat_zero.isel(time=-1) - dm_lonlat_zero.isel(time=0)
totalgre_withnan = dm_lonlat_zero_gre.isel(time=-1) - dm_lonlat_zero_gre.isel(time=0)


# this only works because the values in the totalant are never exactly 0. When this becomes true the code has to be adjusted
maskje = xr.where(
    (np.isnan(totalant_withnan)) | (totalant_withnan == 0),# | np.isnan(totalgre_withnan) | (totalgre_withnan == 0), ## totalant withnan has only nan so I dont need todo this I think
    0,
    1
)

maskjegre = xr.where(
    (np.isnan(totalgre_withnan)) | (totalgre_withnan == 0),
    0,
    1
)

union_mask = xr.where((landsea == 1) |(maskje == 1) | (maskjegre==1), 1, 0)  #(landsea == 1) | 
#union_mask.plot()

### Sea level inpact Antarctica: 

In [8]:
# Option to calculate the total mass loss between 2002 and 2024. 
#totalant = dm_lonlat_zero.isel(time=-1) - dm_lonlat_zero.isel(time=0)


antreference = dm_lonlat_zero - dm_lonlat_zero.isel(time=0)  # calculate the values relative to 2002. 

# Calculate the spatial distribution of the chagnes in sea level due to the mass changes in Antarctica. 
results = []
step=2001
for t in antreference.time:
    step=step+1
    print('Jaar')
    print(step) 
    totalant_t = antreference.sel(time=t)
    sealevel_t = gf.sealevelfunctions(totalant_t, lon_target, lat_target, union_mask)
    results.append(sealevel_t)

# Combine all time slices into a single xarray DataArray
sealevel_all = xr.concat(results, dim=antreference.time)
sealevel_all["time"] = antreference.time

# Make sure that values are nan at land as well

Jaar
2002
Jaar
2003
Jaar
2004
Jaar
2005
Jaar
2006
Jaar
2007
Jaar
2008
Jaar
2009
Jaar
2010
Jaar
2011
Jaar
2012
Jaar
2013
Jaar
2014
Jaar
2015
Jaar
2016
Jaar
2017
Jaar
2018
Jaar
2019
Jaar
2020
Jaar
2021
Jaar
2022
Jaar
2023
Jaar
2024


### Sea level inmpact Greenland

In [9]:
greenreference = dm_lonlat_zero_gre - dm_lonlat_zero_gre.isel(time=0) 

results_gre = []
step=2001
for t in antreference.time:
    step=step+1
    print('Jaar')
    print(step) 
    totalgre_t = greenreference.sel(time=t)
    sealevel_t_gre = gf.sealevelfunctions(totalgre_t, lon_target, lat_target, union_mask)
    results_gre.append(sealevel_t_gre)

# Combine all time slices into a single xarray DataArray
sealevel_all_gre = xr.concat(results_gre, dim=greenreference.time)
sealevel_all_gre["time"] = greenreference.time


# Make sure that values are nan at land as well

Jaar
2002
Jaar
2003
Jaar
2004
Jaar
2005
Jaar
2006
Jaar
2007
Jaar
2008
Jaar
2009
Jaar
2010
Jaar
2011
Jaar
2012
Jaar
2013
Jaar
2014
Jaar
2015
Jaar
2016
Jaar
2017
Jaar
2018
Jaar
2019
Jaar
2020
Jaar
2021
Jaar
2022
Jaar
2023
Jaar
2024


### Calculate mean sea level rise due to melt Antarctica or Greenland

In [10]:
# Create a dataset with sealevelrise where all values at land are NAN: 



# Create a dataset with the area for every cell on the regirdded grid:

# Get the average radius of the eath [cm] using LMAX (same method as in GRD patterns) 
LMAX = 360

factors = gravtk.units(lmax=LMAX)
rad_e = factors.rad_e/10**5 # Convert to km
rad_e_m = rad_e*1000 #convert to meters

area_da = landsea.copy()

deg2rad = np.pi/180
area_d1 = rad_e_m**2*np.cos(area_da.lat*deg2rad)*deg2rad**2   #calculate for every lat and lon combination the area. actually it depends only on latitude 

area_da.data = area_d1.data[:,np.newaxis]*np.ones(len(area_da.lon))

area_da.name = 'cell area [m**2]'



seamask = (union_mask == 0)
weights_masked = area_da.where(seamask)  ## the masked area data 

#weights_masked.plot()
#plt.show()

sealevel_masked = sealevel_all.where(seamask) 
sealevel_masked_gre = sealevel_all_gre.where(seamask) 


# Calculates the mean sea level rise as a result of mass loss in Greenland and Antarctica
antrisemean = (sealevel_masked * weights_masked).sum(dim=['lat', 'lon'], skipna=True) / weights_masked.sum(dim=['lat', 'lon'], skipna=True)
greenrisemean = (sealevel_masked_gre * weights_masked).sum(dim=['lat', 'lon'], skipna=True) / weights_masked.sum(dim=['lat', 'lon'], skipna=True)

#print(antrisemean)
#print(greenrisemean)

### Create .nc file and save the file

In [11]:
# create a nc file with the same format as the data from frederikse to be able to use it easily in the budget 

ant_costg = gf.makefrederiksefile(sealevel_all, antrisemean) 

gre_costg = gf.makefrederiksefile(sealevel_all_gre, greenrisemean)

display(ant_costg)
display(gre_costg)


<xarray.Dataset> Size: 12MB
Dimensions:    (time: 23, lat: 180, lon: 360)
Coordinates:
  * lon        (lon) float64 3kB 0.5 1.5 2.5 3.5 4.5 ... 356.5 357.5 358.5 359.5
  * lat        (lat) float64 1kB -89.5 -88.5 -87.5 -86.5 ... 86.5 87.5 88.5 89.5
  * time       (time) int64 184B 2002 2003 2004 2005 ... 2021 2022 2023 2024
Data variables:
    IS_Gravis  (time, lat, lon) float64 12MB nan nan nan ... 0.7223 0.7202
    Meanslr    (time) float64 184B 0.0 0.02193 0.04769 ... 0.5676 0.5615 0.6
Attributes:
    Info:     Based on Gravis data and created using the gravity toolkit

<xarray.Dataset> Size: 12MB
Dimensions:    (time: 23, lat: 180, lon: 360)
Coordinates:
  * lon        (lon) float64 3kB 0.5 1.5 2.5 3.5 4.5 ... 356.5 357.5 358.5 359.5
  * lat        (lat) float64 1kB -89.5 -88.5 -87.5 -86.5 ... 86.5 87.5 88.5 89.5
  * time       (time) int64 184B 2002 2003 2004 2005 ... 2021 2022 2023 2024
Data variables:
    IS_Gravis  (time, lat, lon) float64 12MB nan nan nan ... -1.081 -1.099
    Meanslr    (time) float64 184B 0.0 0.01419 0.06537 ... 1.187 1.229 1.269
Attributes:
    Info:     Based on Gravis data and created using the gravity toolkit

In [13]:
# Create an output file (netcdf) where the data is saved in a similar format as the data from Frederikse et al. (2020) that is used in the budget. Change the name when doing a different run!

ant_costg.to_netcdf(output_path + 'Gravis_Antarctica.nc')
gre_costg.to_netcdf(output_path + 'Gravis_Greeenland.nc')

